# Interacting with GPIO from MicroBlaze

In [2]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")

In [40]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

In [60]:
write_gpio(1, 1)
print(read_gpio(1))
write_gpio(1,0)
print(read_gpio(1))

1
0


# Multi-tasking with MicroBlaze 

In [1]:
base = BaseOverlay("base.bit")

NameError: name 'BaseOverlay' is not defined

In [3]:
%%microblaze base.PMODA

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODA
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODA
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Multitasking the microblaze for a simple function
int add(int a, int b){
    return a + b;
}

In [4]:
val = 1
write_gpio(0, val)
print(read_gpio(0))
val = 0
write_gpio(0, val)
print(read_gpio(0))

1
0


In [7]:
add(2, 30)

32

# Lab work
Use the code from the second cell as a template and write a code to use two pins (0 and 1) for send and two pins (2 and 3) for receive. You should be able to send 2bits (0~3) over GPIO. You'll need to hardwire from the send pins to the receive pins.

In [54]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")

In [63]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

unsigned int send_val(unsigned int data_in){
    unsigned int a = data_in & 0x1;
    unsigned int b = (data_in >> 1) & 0x1;
    pyprintf("%d",a);
    pyprintf("%d",b);

    write_gpio(0, a);
    write_gpio(1, b);

    unsigned int c = read_gpio(2);
    unsigned int d = read_gpio(3);
    pyprintf("%d",c);
    pyprintf("%d",d);
    unsigned int received = c | (d << 1);

    return received;
}

In [70]:
write_gpio(2,0)
write_gpio(3,0)
for val in range(0,4):
    out = send_val(val)
    print(f"sent: {val}, received: {out}")

0000sent: 0, received: 0
1010sent: 1, received: 1
0101sent: 2, received: 2
1111sent: 3, received: 3
